In [3]:
import requests
from bs4 import BeautifulSoup
import time

In [14]:
import sqlite3

conn = sqlite3.connect('data/bdd.db')
cur = conn.cursor()

########################
# Table equipages
# 🚨🚨 Attention en sqlite 
# on n'utilise pas 'SERIAL' mais 'AUTOINCREMENT NOT NULL'
########################


cur.execute('''
    CREATE TABLE IF NOT EXISTS equipage (
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        nom TEXT,
        n_membres INTEGER
    )
''')
conn.commit()



########################
# Table personnages
########################
cur.execute('''
    CREATE TABLE IF NOT EXISTS personnage (
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        nom TEXT,
        equipage_id INTEGER,
        FOREIGN KEY (equipage_id) REFERENCES equipage(id)

    )
''')
conn.commit()

In [15]:
########################
# Requête initiale + parsing
########################

url = "http://www.volonte-d.com/perso/pirates.php"

r = requests.get(url)
soup = BeautifulSoup(r.text, "html.parser")  
table = soup.find("div", class_="blockCache3").find_all('table')[1]
tables = table.find_all("table")

#######################################
# On itère sur les tables par équipage
######################################

for table in tables:
    ######################################
    # Extraction nom de l'équipage et url relative
    ######################################
    trs = table.find_all("tr")
    href = trs[0].find('a').get("href")
    equipage_nom = trs[1].text
    print(equipage_nom, href)
    
    ######################################
    # On bâtit l'url de chaque équipage
    ######################################
    url_equipage = f"http://www.volonte-d.com/perso/{href}"
    
    
    ######################################
    # Requête équipage
    ######################################
    r_equipage = requests.get(url_equipage)
    soup_equipage = BeautifulSoup(r_equipage.text, "html.parser")  
    
    equipage = [
        p.text 
        for p in soup_equipage.find_all("td", class_="trsbas0")
    ]
    
    ######################################
    # On insère l'équipage
    ######################################
    cur.execute("INSERT INTO equipage (nom, n_membres) VALUES (?, ?)", (equipage_nom, len(equipage)))
    conn.commit()
    
    # 🚨🚨 on récupère l'id de l'équipage 
    # pour l'insertion de la FOREIGN KEY equipage_id à venir
    equipage_id = cur.lastrowid
    
    ######################################
    # On insère les personnages
    ######################################
    for perso_nom in equipage:
        cur.execute("INSERT INTO personnage (nom, equipage_id) VALUES (?, ?)", (perso_nom, equipage_id))
        conn.commit()    

Chapeau de Paille paille.php
Firetank Pirates firetankpirate.php
Bonney Pirates bonneypirate.php
Hawkins Pirates hawkinspirate.php
Kid Pirates kidpirate.php
On-Air Pirates onairpirate.php
Drake Pirates drakepirate.php
Depraved Monk Pirates depravedmonkpirate.php
Heart Pirates heartpirate.php


# Vérification

In [23]:
import pandas as pd

equipage = pd.read_sql("SELECT * from equipage", con=conn)
personnage = pd.read_sql("SELECT * from personnage", con=conn)

equipage.set_index("id").join(personnage.set_index("equipage_id"), rsuffix="equipage")

,nom,n_membres,id,nomequipage
1,Chapeau de Paille,11,1,Monkey D Luffy
1,Chapeau de Paille,11,2,Roronoa Zoro
1,Chapeau de Paille,11,3,Nami
1,Chapeau de Paille,11,4,Usopp
1,Chapeau de Paille,11,5,Sanji
1,Chapeau de Paille,11,6,Tony Tony Chopper
1,Chapeau de Paille,11,7,Nico Robin
1,Chapeau de Paille,11,8,Franky
1,Chapeau de Paille,11,9,Brook
1,Chapeau de Paille,11,10,Jinbe
